In [36]:
# Needed for AWS
# pip install pip install gspread oauth2client
# pip install --upgrade google-api-python-client oauth2client


## Pandas Work

In [42]:
import pandas as pd
from datetime import date
import calendar

In [83]:
df = pd.read_csv('bulk_campaign_stats_export.csv',usecols=['Campaign Name','Date','Requests', 
                                                           'Delivered', 'Unique Opens', 'Unique Clicks',
                                                          'Unsubscribes'])

In [84]:
def category_decide(category_series):
    category_dic = { "MWF":"MWF", "Refund":"Refund","Promo": "National Promo",
                    "MWP":"Field Marketing Promo", 
                    }
    return_list = []
    for row in list(category_series):
        for key in category_dic.keys():
            if key in row:
                return_list.append((category_dic[key]))
    return return_list

In [105]:
#Get every 8th one, which should be 'campaign total'
total_df = df[df['Date'] == 'Campaign Total']
# get the prior dates for the campaigns, index and subtract 1
campaign_dates = (df[df['Date'] == 'Campaign Total'].index-7).values
send_dates_list = df.iloc[campaign_dates]['Date'].tolist()
print(send_dates_list)
send_dates = pd.to_datetime(send_dates_list)
# Now we need to incorporate adding the day of the week
day_names = [day.day_name() for day in send_dates]
#A add in columns
total_df['Date'] = send_dates.strftime('%m/%d/%Y')
total_df['Day Of Week'] = day_names
total_df['Category'] = category_decide(total_df['Campaign Name'])
# #Add in Old Columns with zero
total_df['Opens'] = ""
total_df['Clicks'] = ""
# # reoder columns just for matching up reading
total_df = total_df[['Campaign Name','Date','Day Of Week','Category', 'Requests', 'Delivered','Opens','Unique Opens',
       'Clicks','Unique Clicks', 'Unsubscribes']]

['2018-10-24', '2018-10-29', '2018-11-01']


/home/pipingkid/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/home/pipingkid/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/home/pipingkid/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

In [106]:
total_df['Date']

7     10/24/2018
15    10/29/2018
23    11/01/2018
Name: Date, dtype: object

# Lets try GSpread

## And for something simpler

Note: <b>The client email needs to be given access to the sheet as an editor for read/write permissions.</b> 
-Ensure that creds is avalible to get the token json for HTTP authentication.
-Writing is realitively simple with each row can be crafted. This should be done via pandas.

In [107]:
new_entries = total_df.values.tolist()

In [108]:
new_entries[0][1]

'10/24/2018'

In [111]:
#Access the worksheet
## Must have client_email shared with in the google sheet itself!
import gspread
from oauth2client.service_account import ServiceAccountCredentials

credentials = ServiceAccountCredentials.from_json_keyfile_name(
                'client_secret.json',
                ['https://spreadsheets.google.com/feeds'])
client = gspread.authorize(credentials)

    #Put in the specific google sheet URL to access. MUST BE SHARED WITH THE CLIENT ID FROM CREDENTIALS
sheet = client.open_by_url('https://docs.google.com/spreadsheets/d/1pUiI-Bjhuv96P9kxs4ZYWtyqkbSryreLGCh-tof1Oto/edit?usp=sharing')
worksheet = sheet.get_worksheet(0)


In [112]:
for i in range(len(new_entries)):
    worksheet.append_row(new_entries[i],value_input_option='USER_ENTERED')